In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("/content/drive/My Drive/CASE Studies/CASE1/PE_data.csv")
df.shape

(13139, 15)

In [4]:
df.head()

,Unnamed: 0,Price,departure_time,arrival_time,Airline,Cabin,Dept_city,Dept_date,arrival_city,stops,duration,weekday,dept_hours,Dept_flights_time,optimal_hours
0,317799,6855,18:55,21:35,Spicejet,PE,New Delhi,10,Chennai,0,160,0,18,Evening_flight,2
1,317800,12168,19:10,13:15,Spicejet,PE,New Delhi,10,Chennai,1,1085,0,19,Evening_flight,3
2,317801,10096,19:10,20:25,Spicejet,PE,New Delhi,10,Guwahati,1,75,0,19,Evening_flight,3
3,317802,11788,19:10,20:50,Spicejet,PE,New Delhi,10,Patna,0,100,0,19,Evening_flight,3
4,317803,6147,19:45,21:55,Vistara,PE,Mumbai,10,New Delhi,0,130,0,19,Evening_flight,3


In [5]:
df.drop(["Unnamed: 0"],axis=1,inplace=True)
df.columns

Index(['Price', 'departure_time', 'arrival_time', 'Airline', 'Cabin',
       'Dept_city', 'Dept_date', 'arrival_city', 'stops', 'duration',
       'weekday', 'dept_hours', 'Dept_flights_time', 'optimal_hours'],
      dtype='object')

In [6]:
df.shape

(13139, 14)

In [7]:
df.drop(columns=["departure_time","arrival_time","Cabin"],inplace=True,axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13139 entries, 0 to 13138
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Price              13139 non-null  int64 
 1   Airline            13139 non-null  object
 2   Dept_city          13139 non-null  object
 3   Dept_date          13139 non-null  int64 
 4   arrival_city       13139 non-null  object
 5   stops              13139 non-null  int64 
 6   duration           13139 non-null  int64 
 7   weekday            13139 non-null  int64 
 8   dept_hours         13139 non-null  int64 
 9   Dept_flights_time  13139 non-null  object
 10  optimal_hours      13139 non-null  int64 
dtypes: int64(7), object(4)
memory usage: 1.1+ MB


In [8]:
for i in range(df.shape[0]):
  if df['optimal_hours'].iloc[i]<0:
    
    df['optimal_hours'].iloc[i] = 24 + df['optimal_hours'].iloc[i]
    # print(df['optimal_hours'].iloc[i])

In [9]:
# only using single airline flights
ind=[]
for i in range(df.shape[0]):
  if "," in df["Airline"].iloc[i]:
    ind.append(i)
df.drop(ind,inplace=True,axis=0)

In [10]:
air=df["Airline"].value_counts().sort_values(ascending=False)
air
l1 = []
l1.append(df[df["Airline"] == "flybig"].index)
l1.append(df[df["Airline"] == "TruJet"].index)
l1.append(df[df["Airline"] == "Air India Express"].index)
#print(l1)
l2 = []
for i in l1:
    for j in i:
        l2.append(j)
df.drop(l2,inplace=True,axis=0)

In [11]:
dff = df.copy()

In [12]:
x=["Airline","arrival_city","Dept_city","Dept_flights_time"]
from sklearn.preprocessing import LabelEncoder
for i in x:
  encoder=LabelEncoder()
  df[i]=encoder.fit_transform(df[[i]])

In [13]:
df.dropna(inplace=True,how='any')

In [14]:
ind4 = list(df[df.stops==4].index)
ind5 = list(df[df.stops==5].index)
ind3 = list(df[df.stops==3].index)
df.drop(ind4,inplace=True,axis=0)
df.drop(ind5,inplace=True,axis=0)
df.drop(ind3,inplace=True,axis=0)
df.shape

(13126, 11)

In [15]:
df.sort_values(by=["Dept_date"],ascending=True,inplace=True)

In [16]:
# optimal hours
x=df.drop(["Price","optimal_hours"],axis=1)
y1=df["Price"]
y2=df["optimal_hours"]

import xgboost as xg
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y2,test_size=0.5,random_state=0)

scale_cols=["Dept_date","duration","dept_hours"]

from sklearn.preprocessing import MinMaxScaler
ss=MinMaxScaler()
from sklearn.externals.joblib import dump, load
def scale(train,test,col):
  
  ss.fit(train[col].values.reshape(-1,1))
  dump(ss, '/content/drive/My Drive/CASE Studies/CASE1/Scaling/PE_'+col+'_std_scaler.bin', compress=True)

  s_tr=ss.transform(train[col].values.reshape(-1,1))
  s_te=ss.transform(test[col].values.reshape(-1,1))

  return s_tr,s_te

for i in scale_cols:
  x_train[i],x_test[i]=scale(x_train,x_test,i) 


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
params={'max_depth':[5,10,15,20,50],
        'min_samples_split':[2,3,5,10]}
rfg_time=RandomForestRegressor(n_estimators=100)
cv_time=RandomizedSearchCV(rfg_time,params,cv=5,n_jobs=-1,verbose=2)
cv_time.fit(x_train,y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   42.4s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [17]:
print(cv_time.best_params_)
model_time=cv_time.best_estimator_
model_time.fit(x_train,y_train)

pred = model_time.predict(x_test)

from sklearn.metrics import mean_squared_error as mse , r2_score
from math import sqrt
rms = sqrt(mse(y_test, pred))
score = r2_score(y_test, pred)
print(rms)
print(score)

xxx = pd.DataFrame({'Actual time': y_test, 'Predicted time': pred})
xxx.head(5)

{'min_samples_split': 2, 'max_depth': 20}
2.9847005071539185
0.8188299502091815


,Actual time,Predicted time
4344,22,21.980000
1648,8,9.041464
10903,11,10.995246
11528,0,17.092500
12053,1,1.364444


In [18]:
import pickle as pkl
with open("/content/drive/My Drive/CASE Studies/CASE1/PE_time_predict.pkl","wb") as f:
  pkl.dump(model_time,f)

In [19]:
x_test["dept_hours"]=pred
for i in x_test.index:
  x_test["dept_hours"][i]=int(x_test["dept_hours"][i])
x_test.head()

y_test_=[]
for i in x_test.index:
  y_test_.append(y1[i])
y_test_=np.asarray(y_test_)

In [20]:
x_test.shape ,y_test_.shape

((6563, 9), (6563,))

In [21]:
# price
#x=df.drop(["Price","dept_hours"],axis=1)
#y1=df["Price"]

import xgboost as xg
from sklearn.model_selection import train_test_split
x_tr,x_ts,y_tr,y_ts=train_test_split(x_test,y_test_,test_size=0.2,random_state=0)

scale_cols=["dept_hours"]

from sklearn.preprocessing import MinMaxScaler
ss=MinMaxScaler()
from sklearn.externals.joblib import dump, load
def scale(train,test,col):
  
  ss.fit(train[col].values.reshape(-1,1))
  dump(ss, '/content/drive/My Drive/CASE Studies/CASE1/Scaling/PE_price_dept_hours_std_scaler.bin', compress=True)

  s_tr=ss.transform(train[col].values.reshape(-1,1))
  s_te=ss.transform(test[col].values.reshape(-1,1))

  return s_tr,s_te

for i in scale_cols:
  x_tr[i],x_ts[i]=scale(x_tr,x_ts,i) 

from sklearn.ensemble import RandomForestRegressor
params={'max_depth':[5,10,15,20,50],
        'min_samples_split':[2,3,5,10]}
rfg_price=RandomForestRegressor(n_estimators=100)
cv_price=RandomizedSearchCV(rfg_price,params,cv=5,n_jobs=-1,verbose=2)
cv_price.fit(x_tr,y_tr)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   40.4s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [22]:
print(cv_price.best_params_)
model_price=cv_time.best_estimator_
model_price.fit(x_tr,y_tr)

pred = model_price.predict(x_ts)
from sklearn.metrics import mean_squared_error as mse , r2_score
from math import sqrt
rms = sqrt(mse(y_ts, pred))
score = r2_score(y_ts, pred)
print(rms)
print(score)

xxx = pd.DataFrame({'Actual price': y_ts, 'Predicted price': pred})
xxx.head(5)

{'min_samples_split': 2, 'max_depth': 50}
825.4604654691344
0.8055487899574578


,Actual price,Predicted price
0,7743,7475.646503
1,5669,6015.720000
2,6686,6700.750028
3,7918,7895.190000
4,8009,7791.050262


In [23]:
import pickle as pkl
with open("/content/drive/My Drive/CASE Studies/CASE1/PE_price_predict.pkl","wb") as f:
  pkl.dump(model_price,f)

In [24]:
a=df["Dept_city"].value_counts().keys()
b=dff["Dept_city"].value_counts().keys()
D_city=dict()
for i in range(len(a)):
  D_city[b[i].strip().lower()]=a[i]

print(D_city)
with open('/content/drive/My Drive/CASE Studies/CASE1/PE_dept_city_dict.pkl',"wb") as f:
  pkl.dump(D_city,f)

{'new delhi': 17, 'mumbai': 16, 'goa': 7, 'chennai': 5, 'patna': 18, 'bengaluru': 2, 'srinagar': 23, 'guwahati': 8, 'bagdogra': 1, 'hyderabad': 9, 'pune': 20, 'kolkata': 13, 'amritsar': 0, 'varanasi': 25, 'jaipur': 11, 'kochi': 12, 'mangalore': 15, 'ranchi': 22, 'bhubaneswar': 3, 'port blair': 19, 'chandigarh': 4, 'visakhapatnam': 26, 'indore': 10, 'raipur': 21, 'lucknow': 14, 'thiruvananthapuram': 24, 'coimbatore': 6}


In [25]:
a=df["Airline"].value_counts().keys()
b=dff["Airline"].value_counts().keys()
airline=dict()
for i in range(len(a)):
  airline[a[i]]=b[i].strip().lower()

print(airline)
with open('/content/drive/My Drive/CASE Studies/CASE1/PE_airline_dict.pkl',"wb") as f:
  pkl.dump(airline,f)

{0: 'spicejet', 1: 'vistara'}


In [26]:
a=df["arrival_city"].value_counts().keys()
b=dff["arrival_city"].value_counts().keys()
A_city=dict()
for i in range(len(a)):
  A_city[b[i].strip().lower()]=a[i]
print(A_city)
with open('/content/drive/My Drive/CASE Studies/CASE1/PE_arrival_city_dict.pkl',"wb") as f:
  pkl.dump(A_city,f)


{'new delhi': 17, 'mumbai': 16, 'goa': 7, 'chennai': 5, 'guwahati': 8, 'patna': 18, 'bagdogra': 1, 'kolkata': 13, 'bengaluru': 2, 'srinagar': 23, 'pune': 20, 'amritsar': 0, 'hyderabad': 9, 'varanasi': 25, 'mangalore': 15, 'port blair': 11, 'jaipur': 19, 'ranchi': 22, 'bhubaneswar': 3, 'kochi': 12, 'chandigarh': 4, 'visakhapatnam': 26, 'raipur': 21, 'thiruvananthapuram': 24, 'lucknow': 14, 'indore': 10, 'coimbatore': 6}


In [28]:
dcity=input("Enter the Departure ciity : ")
acity=input("Enter the Arrival ciity: ")
date=input("Enter the date of of the journey (YYYY-MM_DD): ")
time=int(input("Enter booking time: "))
# cab=input("Enter the Cabin: ")

Enter the Departure ciity : new delhi
Enter the Arrival ciity: mumbai
Enter the date of of the journey (YYYY-MM_DD): 2021-06-01
Enter booking time: 5


In [29]:
a=df["Dept_flights_time"].value_counts().keys()
b=dff["Dept_flights_time"].value_counts().keys()
flight_time=dict()
for i in range(len(a)):
  flight_time[b[i]]=a[i]
print(flight_time)
with open('/content/drive/My Drive/CASE Studies/CASE1/PE_flight_time_dict.pkl',"wb") as f:
  pkl.dump(flight_time,f)

# for i in range(df.shape[0]):
if time>=16 and time<21:
  temp_time = 1
      # d.append("Evening_flight")
    
if time>=21 or time<5:
  temp_time = 3
      # d.append("Night_flight")
    
if time>=5 and time<11:
  temp_time = 2
      # d.append("Morning_flight")
    
if time>=11 and time<16:
  temp_time = 0
      # d.append("Afternoon_flight")


{'Evening_flight': 1, 'Morning_flight': 2, 'Afternoon_flight': 0, 'Night_flight': 3}


In [31]:
month=pd.to_datetime(date).month
hour=time
day=pd.to_datetime(date).day
weekday=pd.to_datetime(date).dayofweek
dept_flight_time = temp_time


In [33]:
day=np.asarray(day)
hour=np.asarray(hour)

sc=load('/content/drive/My Drive/CASE Studies/CASE1/Scaling/PE_Dept_date_std_scaler.bin')
Dept_date=sc.transform(day.reshape(-1,1))

sc=load('/content/drive/My Drive/CASE Studies/CASE1/Scaling/PE_dept_hours_std_scaler.bin')
dept_hours=sc.transform(hour.reshape(-1,1))

with open('/content/drive/My Drive/CASE Studies/CASE1/duration.pkl',"rb") as f:
  duration=pkl.load(f)

k_0=dcity+","+acity+" ,"+str(0)
k_1=dcity+","+acity+" ,"+str(1)
k_0=np.asarray(duration[k_0])
k_1=np.asarray(duration[k_1])

sc=load('/content/drive/My Drive/CASE Studies/CASE1/Scaling/duration_std_scaler.bin')
scale_0=sc.transform(k_0.reshape(-1,1))
sc=load('/content/drive/My Drive/CASE Studies/CASE1/Scaling/duration_std_scaler.bin')
scale_1=sc.transform(k_1.reshape(-1,1))



import pickle
with open("/content/drive/My Drive/CASE Studies/CASE1/PE_time_predict.pkl","rb") as f:
  model_time=pkl.load(f)

from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Airline", "Stops", "Source_city", "Arrival_city","Duration","Optimal_time"]
for j in range(2):
  p=model_time.predict([[ j ,D_city[dcity],Dept_date, A_city[acity] ,0,scale_0 ,weekday,dept_hours,temp_time]])
  x.add_row([airline[j] ,0, dcity,acity ,k_0, p[0]])
for j in range(2):
  p=model_time.predict([[ j ,D_city[dcity],Dept_date, A_city[acity] ,1,scale_1 ,weekday,dept_hours,temp_time]])
  x.add_row([airline[j] ,1, dcity,acity ,k_1, p[0]])

print(x)


+----------+-------+-------------+--------------+----------+--------------+
| Airline  | Stops | Source_city | Arrival_city | Duration | Optimal_time |
+----------+-------+-------------+--------------+----------+--------------+
| spicejet |   0   |  new delhi  |    mumbai    |  125.0   |    10.47     |
| vistara  |   0   |  new delhi  |    mumbai    |  125.0   |    10.54     |
| spicejet |   1   |  new delhi  |    mumbai    |  625.0   |    10.45     |
| vistara  |   1   |  new delhi  |    mumbai    |  625.0   |    10.51     |
+----------+-------+-------------+--------------+----------+--------------+


In [34]:
sc=load('/content/drive/My Drive/CASE Studies/CASE1/Scaling/PE_price_dept_hours_std_scaler.bin')
dept_hours=sc.transform(p.reshape(-1,1))
import pickle
with open("/content/drive/My Drive/CASE Studies/CASE1/PE_price_predict.pkl","rb") as f:
  model_price=pickle.load(f)

from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Airline", "Stops", "Source_city", "Arrival_city","Duration","Optimal_time","Price"]
for j in range(2):
  pred=model_price.predict([[ j ,D_city[dcity],Dept_date, A_city[acity] ,0,scale_0 ,weekday,dept_hours,temp_time]])
  x.add_row([airline[j] ,0, dcity,acity,k_0 ,p[0] ,pred[0]])

for j in range(2):
  pred=model_price.predict([[ j ,D_city[dcity],Dept_date, A_city[acity] ,1,scale_1 ,weekday,dept_hours,temp_time]])
  x.add_row([airline[j] ,1, dcity,acity,k_1 ,p[0] ,pred[0]])

print(x)

+----------+-------+-------------+--------------+----------+--------------+-------------------+
| Airline  | Stops | Source_city | Arrival_city | Duration | Optimal_time |       Price       |
+----------+-------+-------------+--------------+----------+--------------+-------------------+
| spicejet |   0   |  new delhi  |    mumbai    |  125.0   |    10.51     |      5872.38      |
| vistara  |   0   |  new delhi  |    mumbai    |  125.0   |    10.51     |      5831.35      |
| spicejet |   1   |  new delhi  |    mumbai    |  625.0   |    10.51     |      7659.47      |
| vistara  |   1   |  new delhi  |    mumbai    |  625.0   |    10.51     | 8709.931905920905 |
+----------+-------+-------------+--------------+----------+--------------+-------------------+
